I had to clasify the English texts and singlish texts by cheking the stop words, but it will not be successfull when commenters add singlish commet with some english stop words


So just decide to train a model

In [86]:
import pandas as pd

In [87]:
english_df = pd.read_csv('/content/drive/MyDrive/Dataset/Anonote/finalized_data/english_texts_partion.csv')

In [88]:
english_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   300 non-null    object
dtypes: object(1)
memory usage: 2.5+ KB


In [89]:
english_df.head()

,tweet
0,why be it everytime I go to cracker barrel the...
1,run that nigga you do not want that nigga but ...
2,I need a girl from Jamaica I can not fuck with...
3,RT I know you have I block but next week bitch
4,put ya hand up if you a Grade A bitch


In [90]:
english_df['Comment'] = english_df['tweet']
english_df.drop(columns=['tweet'], inplace=True)

In [91]:
english_df['E/S'] = 1

In [92]:
singlish_df = pd.read_csv('/content/drive/MyDrive/Dataset/Anonote/finalized_data/Singlish_Comments_v2.csv')

In [93]:
singlish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Comment            403 non-null    object
 1   positive/negative  403 non-null    object
dtypes: object(2)
memory usage: 6.4+ KB


In [94]:
singlish_df.drop(columns=['positive/negative'], inplace=True)

In [95]:
singlish_df = singlish_df.sample(n=300, random_state=42)

In [96]:
singlish_df['E/S'] = 0

In [97]:
df = pd.concat([english_df, singlish_df], ignore_index=True)

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  600 non-null    object
 1   E/S      600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB


In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import pickle

In [100]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Comment'],
    df['E/S'],
    test_size=0.2,
    random_state=42,
    stratify=df['E/S'],
)

In [101]:
clf = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        60

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



In [102]:
path = '/content/drive/MyDrive/Dataset/Anonote/models/english_singlish_classifier_v1.pkl'

with open(path, 'wb') as file:
    pickle.dump(clf, file)

In [104]:
clf.predict(['hi api muge nahaya kamu'])

array([0])